In [ ]:
import theano
import theano.tensor as T
import numpy as np

In [ ]:
class LSTM_cell(object):
    
    
    def __init__(self):
        self.random_state = np.random.RandomState(284783)
        self.output_dim = 10
        self.outer_output_dim = 1
        self.input_dim = self.outer_output_dim
        self.layer_id = "0"
        self.learning_rate = 0.1
        
        self.initialize_params()
        
       
        
        
    def initialize_params(self):
        U_input = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     (self.output_dim, self.input_dim))
            , dtype=theano.config.floatX)

        U_forget = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     (self.output_dim, self.input_dim))
            , dtype=theano.config.floatX)

        U_output = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     (self.output_dim, self.input_dim))
            , dtype=theano.config.floatX)

        W_input = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     (self.output_dim, self.output_dim))
            , dtype=theano.config.floatX)

        W_forget = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     (self.output_dim, self.output_dim))
            , dtype=theano.config.floatX)

        W_output = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     (self.output_dim, self.output_dim))
            , dtype=theano.config.floatX)

        U = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.input_dim + self.output_dim)),
                                     (self.output_dim, self.input_dim))
            , dtype=theano.config.floatX)

        W = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                     (self.output_dim, self.output_dim))
            , dtype=theano.config.floatX)

        bias_input = np.zeros(self.output_dim, dtype=theano.config.floatX)
        bias_forget = np.zeros(self.output_dim, dtype=theano.config.floatX)
        bias_output = np.zeros(self.output_dim, dtype=theano.config.floatX)
        bias = np.zeros(self.output_dim, dtype=theano.config.floatX)

        
        self.W = theano.shared(value=W, name="W" + self.layer_id, borrow="True")
        self.U = theano.shared(value=U, name="U" + self.layer_id, borrow="True")
        self.bias = theano.shared(value=bias, name="bias", borrow="True")

        self.W_input = theano.shared(value=W_input, name="W_input" + self.layer_id, borrow="True")
        self.U_input = theano.shared(value=U_input, name="U_input" + self.layer_id, borrow="True")
        self.bias_input = theano.shared(value=bias_input, name="bias_input" + self.layer_id, borrow="True")

        self.W_output = theano.shared(value=W_output, name="W_output" + self.layer_id, borrow="True")
        self.U_output = theano.shared(value=U_output, name="U_output" + self.layer_id, borrow="True")
        self.bias_output = theano.shared(value=bias_output, name="bias_output" + self.layer_id, borrow="True")

        self.W_forget = theano.shared(value=W_forget, name="W_forget" + self.layer_id, borrow="True")
        self.U_forget = theano.shared(value=U_forget, name="U_forget" + self.layer_id, borrow="True")
        self.bias_forget = theano.shared(value=bias_forget, name="bias_forget" + self.layer_id, borrow="True")
        self.W = theano.shared(value=W, name="W" + self.layer_id, borrow="True")
        self.U = theano.shared(value=U, name="U" + self.layer_id, borrow="True")
        self.bias = theano.shared(value=bias, name="bias", borrow="True")
        O_w = np.asarray(
            self.random_state.normal(-np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                      np.sqrt(6.0 / (self.output_dim + self.output_dim)),
                                      (self.outer_output_dim,self.output_dim))
            , dtype=theano.config.floatX)

        O_bias = np.zeros(self.outer_output_dim, dtype=theano.config.floatX)

        self.O_w = theano.shared(value=O_w, name="O_w"+self.layer_id, borrow="True")
        self.O_bias = theano.shared(value=O_bias, name="O_bias"+self.layer_id, borrow="True")

        self.params = [self.U_input, self.U_forget, self.U_output, self.W_input, self.W_forget, self.W_output,
                       self.bias_input, self.bias_forget, self.bias_output, self.U, self.W, self.bias,
                       ]

        self.output_params = [self.O_w,self.O_bias]
    

    
    def buil_model(self):
        def forward_step(x_t,prev_state,prev_content):
            input_gate = T.nnet.hard_sigmoid(T.dot(self.U_input,x_t) + T.dot(self.W_input,prev_state) + self.bias_input)
            forget_gate = T.nnet.hard_sigmoid(T.dot(self.U_forget,x_t) + T.dot(self.W_forget,prev_state) + self.bias_forget)
            output_gate = T.nnet.hard_sigmoid(T.dot(self.U_output,x_t) + T.dot(self.W_output,prev_state) + self.bias_output)

            stabialized_input = T.tanh(T.dot(self.U,x_t) + T.dot(self.W,prev_state) + self.bias)
            c = forget_gate * prev_content + input_gate * stabialized_input
            s = output_gate * T.tanh(c)

            o = T.nnet.sigmoid(T.dot(self.O_w,s) + self.O_bias)

            return [o,s,c]
        
        params = self.params + self.output_params
        
        initial_hidden_state = T.matrix('H')
        y = T.matrix('Y')
        x = T.matrix('X')    
        
        [output,states,mem_content],updates = theano.scan(forward_step, sequences=x,
                                                          outputs_info=[None, 
                                                                        dict(initial=T.zeros(self.output_dim,dtype=theano.config.floatX)), 
                                                                        dict(initial=T.zeros(self.output_dim,dtype=theano.config.floatX))])
        
        self.forward_step = theano.function([x],[output])
        
       
        cost = T.argmax(abs( y - np.round(output)))
        grads = theano.grad(cost,params)
        updates = [ (p,p - self.learning_rate*g*p) for p,g in zip(params,grads)]
        self.training_step = theano.function([x,y],[output], updates=updates)
        
        
            
    
    



    

In [ ]:
lstm_cell = LSTM_cell()

In [ ]:

for i in np.arange(100):
    H = np.random.binomial(1, np.random.random(), size=10)
    Y = [[h] for h in H]
    X = [[0] for y in Y]
    [outputs] = lstm_cell.training_step(X,Y)
    print(outputs)



In [ ]:
lstm_cell.buil_model()
print(np.argmax(outputs))

In [ ]:
print(outputs)
print(np.argmax(outputs,axis=1))